In [64]:
import pandas as pd
import numpy as np 
import networkx as nx
import scipy.stats as stats

In [65]:


def random_graph(prob_resp,prob_nuevo):
    """
    prob_resp float    define la probabilidad de que al tomar un nodo este haya sido un respondiente de la encuesta
    prob_nuevo float   define la probabilidad de que un respondiente tenga conexiones con nodos nuevos

    Esta funcion genera graficas aleatorias basadas en el metodo con el que se crearon las networks de emprendimiento
    """
    G=nx.Graph()
    queue=["0"]
    max_index=0
    max_respondents=random.randint(20,30)             #Indicador que nos dice cuantos respondientes puede haber
    respondents=0
    while len(queue)!=0:
        if (random.random()<prob_resp and respondents<max_respondents) or queue[0]=="0":      #Crear conexiones para un respondiente
            respondents+=1
            n=random.randint(6,21)
            new_n=0
            old_n=0
            for i in range(n):                                              #Calcular cuantas conexiones son con nodos nuevos y cuantos con ya existentesz
                if random.random()<prob_nuevo:
                    new_n+=1
                else:
                    old_n+=1
            for i in range(new_n):
                new_node=str(i+max_index+1)
                queue.append(new_node)
                G.add_node(new_node)
                G.add_edge(queue[0],new_node)
            for i in range(old_n):
                node=str(random.randint(0,int(queue[0]))-1)
                G.add_edge(queue[0],node)
            max_index+=new_n
            queue.pop(0)
        else:                                                                #Crear conexiones para los no respondientes
            if random.random()<0.25:
                n=random.randint(1,6)
                for i in range(n):
                    node=str(random.randint(0,int(queue[0]))-1)
                    G.add_edge(queue[0],node)
            queue.pop(0)
    return G
    
G1 = random_graph(0.25,0.55)
G2 = random_graph(0.25,0.55)
G3 = random_graph(0.25,0.55)
G4 = random_graph(0.25,0.55)
G5 = random_graph(0.25,0.55)
G6 = random_graph(0.25,0.55)
G7 = random_graph(0.25,0.55)
G8 = random_graph(0.25,0.55)
G9 = random_graph(0.25,0.55)
G10 = random_graph(0.25,0.55)
#nx.write_graphml(G1,'Random_Graphs/Random_Graph_1.graphml')
#nx.write_graphml(G2,'Random_Graphs/Random_Graph_2.graphml')
#nx.write_graphml(G3,'Random_Graphs/Random_Graph_3.graphml')
#nx.write_graphml(G4,'Random_Graphs/Random_Graph_4.graphml')
#nx.write_graphml(G5,'Random_Graphs/Random_Graph_5.graphml')
#nx.write_graphml(G6,'Random_Graphs/Random_Graph_6.graphml')
#nx.write_graphml(G7,'Random_Graphs/Random_Graph_7.graphml')
#nx.write_graphml(G8,'Random_Graphs/Random_Graph_8.graphml')
#nx.write_graphml(G9,'Random_Graphs/Random_Graph_9.graphml')
#nx.write_graphml(G10,'Random_Graphs/Random_Graph_10.graphml')

In [66]:
from numpy.random import choice

def are_adjacent(u,v,G):
    if v not in G.nodes():
        return False
    elif u in G.neighbors(v):
        return True
    elif v in G.neighbors(u):
        return True
    else:
        return False

def random_graph_2(n_evals, prob_out, prob_new):
    '''
    Generator of a random graph, given that n
    Given a specific number of respondents of the questionnaire, and that each of them could have provided 25 responses maximum
    
    Input:
     - n_evals  : number of evaluators responding questionnaire
     - prob_out : probability that a mentionned collaboration is outside of the network of evaluators
    
    '''
    
    G=nx.DiGraph()
    setedges = set()
    
    inside = dict()
    outside = dict()
    
    last_index = n_evals-1
    
    for i in range(n_evals):              #add all evaluators to the graph
        G.add_node(i)
        inside[i] = 0
        
    # evaluator |--> no. of collaborations,   in (0,25)~ normal distribution
    sample_num_evaluations = np.random.normal(loc=12.3, scale=4, size=n_evals)
    list_degrees_evaluators = [int(x) for x in sample_num_evaluations]
    
    for i in inside.keys():
        for j in range(list_degrees_evaluators[i]):
            
            if random.random()<prob_out:           #edge added outside evaluators' list
                if outside == dict():                              # if list outside evaluators is new
                    last_index+=1
                    to = last_index
                    G.add_node(to)
                    G.add_edge(i,to)
                    setedges.add((i,to))
                    outside[to] = 1
                    inside[i] += 1
                else:
                    if random.random()<prob_new:               # if edge goes to a new vertex
                        last_index+=1       
                        to=last_index
                        G.add_node(to)
                        G.add_edge(i,to)
                        setedges.add((i,to))
                        outside[to] = 1
                        inside[i] +=1
                        
                    else:
                        vertices, degrees = [], []
                        for vertex, degree in outside.items():
                            if vertex!=i:
                                vertices.append(vertex)
                                degrees.append(degree+1)
                        s = sum(degrees)
                        w = [x/s for x in degrees]
                        to = choice(vertices, size=1, p=w)[0]
                        #while (i,to) in G.edges():
                        #while are_adjacent(i,to,G):
                        #    to = choice(vertices, size=1, p=w)
                        G.add_node(to)
                        G.add_edge(i,to)
                        setedges.add((i,to))
                        outside[to]+=1
                        inside[i]+=1
                        
            else:
                vertices, degrees = [], []
                for vertex, degree in inside.items():
                    if vertex != i:
                        vertices.append(vertex)
                        degrees.append(degree+1)
                s = sum(degrees)
                w = [x/s for x in degrees]
                to = choice(vertices, size=1, p=w)[0]
                #while (i,to) in G.edges():
                #while are_adjacent(i,to,G):
                #    to = choice(vertices, size=1, p=w)
                G.add_node(to)
                G.add_edge(i,to)
                setedges.add((i,to))
                inside[to]+=1
                inside[i]+=1
                
    #print(n_evaluated_list)
    #print('')
    #print(G.nodes())
    #print('')
    #print(G.edges())
    #print('')
    #print([len(list(G.neighbors(x))) for x in G.nodes()])
    return G         


list_cities = ['Aguascalientes', 'Buenos Aires', 'Ciudad de México', 'Guadalajara', 'Hidalgo',
                  'Madrid', 'Montevideo', 'Oaxaca', 'Sao Paulo', 'Santiago de Chile']
num_evaluators = {'Aguascalientes':19, 'Buenos Aires':31, 'Ciudad de México':36, 'Guadalajara':32, 'Hidalgo':19,
                  'Madrid':37, 'Montevideo':48, 'Oaxaca':36, 'Sao Paulo':28, 'Santiago de Chile':25}
for city in list_cities:
    G= random_graph_2(num_evaluators[city],0.2, 0.5)
    nx.write_graphml(G,'../Random_Graphs_Second_Type_Corrected/Random_Graph_'+city+'.graphml')


In [67]:
muestra = pd.read_csv('../Data_answers_evaluators/Evaluadores ecosistemas.xlsx - Muestra.csv')             


ags_info=pd.read_csv('../Data_answers_evaluators/Evaluadores ecosistemas.xlsx - AGS.csv')             
caba_info=pd.read_csv('../Data_answers_evaluators/Evaluadores ecosistemas.xlsx - CABA.csv')             
cdmx_info=pd.read_csv('../Data_answers_evaluators/Evaluadores ecosistemas.xlsx - CDMX.csv')             
gdl_info=pd.read_csv('../Data_answers_evaluators/Evaluadores ecosistemas.xlsx - GDL.csv')             
hgo_info=pd.read_csv('../Data_answers_evaluators/Evaluadores ecosistemas.xlsx - HGO.csv')             
mad_info=pd.read_csv('../Data_answers_evaluators/Evaluadores ecosistemas.xlsx - MAD.csv')             
mtv_info=pd.read_csv('../Data_answers_evaluators/Evaluadores ecosistemas.xlsx - MVD.csv')             
oax_info=pd.read_csv('../Data_answers_evaluators/Evaluadores ecosistemas.xlsx - OAX.csv')             
sao_info=pd.read_csv('../Data_answers_evaluators/Evaluadores ecosistemas.xlsx - SAO.csv')             
scl_info=pd.read_csv('../Data_answers_evaluators/Evaluadores ecosistemas.xlsx - SCL.csv')  

ags_info.rename(columns={'Unnamed: 1':'Colabs'}, inplace=True)
gdl_info.rename(columns={'Unnamed: 1':'Colabs'}, inplace=True)
hgo_info.rename(columns={'Unnamed: 1':'Colabs'}, inplace=True)
oax_info.rename(columns={'Unnamed: 1':'Colabs'}, inplace=True)


list_cities = ['AGS', 'CABA', 'CDMX', 'GDL', 'Pachuca',
                  'Madrid', 'Montevideo', 'Oaxaca', 'Sao Paulo', 'Santiago']
cities_info = {'AGS': ags_info, 
              'CABA' : caba_info,
              'CDMX' : cdmx_info,
              'GDL' : gdl_info,
              'Pachuca' : hgo_info,
              'Madrid' : mad_info,
              'Montevideo' : mtv_info,
              'Oaxaca' : oax_info,
              'Sao Paulo' : sao_info,
              'Santiago' : scl_info}

In [68]:
#(muestra['GDL'][0])
muestra
#gdl_info['Colabs'][31]
#sum(list(gdl_info['Colabs']))

,Unnamed: 0,Madrid,Madrid SG,Madrid Typeform,CDMX,Santiago,CABA,Sao Paulo,Montevideo,Oaxaca,GDL,Pachuca,AGS
0,Muestra,38,16,24,51,30,36,34,59,36,32,21,19
1,Nodes,239,120,156,299,195,228,216,198,149,187,125,96
2,Edges,453,166,205,547,385,500,364,767,326,474,254,233
3,Promedio colabs. por participante,12.13,10.13,8.63,12.33,13.04,13.48,10.38,13.4,6.39,10.81,9.52,8.21
4,% Muestra,15.90%,13.33%,15.38%,17.06%,15.38%,15.79%,15.74%,29.80%,24.16%,17.11%,16.80%,19.79%


In [69]:
list_cities_2 = ['AGS', 'CABA', 'CDMX', 'GDL', 'Pachuca',
                  'Madrid', 'Montevideo', 'Oaxaca', 'Sao Paulo', 'Santiago']
cities_info_2 = {'AGS': ags_info, 
              'CABA' : caba_info,
              'CDMX' : cdmx_info,
              'GDL' : gdl_info,
              'Pachuca' : hgo_info,
              'Madrid' : mad_info,
              'Montevideo' : mtv_info,
              'Oaxaca' : oax_info,
              'Sao Paulo' : sao_info,
              'Santiago' : scl_info}

'''
for city in list_cities_2:
    print(city)
    M = int(muestra[city][0])
    L = len(list(cities_info_2[city]['Colabs']))
    S = sum(list(cities_info_2[city]['Colabs']))
    
    print( M )
    print( L )
    print( S )
    print( M*2 + S )
    print('')
'''

"\nfor city in list_cities_2:\n    print(city)\n    M = int(muestra[city][0])\n    L = len(list(cities_info_2[city]['Colabs']))\n    S = sum(list(cities_info_2[city]['Colabs']))\n    \n    print( M )\n    print( L )\n    print( S )\n    print( M*2 + S )\n    print('')\n"

In [83]:
def check_graph(G):
    F=G.copy()
    for x in G.nodes():
        if G.degree(x) == 0:
            F.remove_node(x)
    return F

In [86]:
def random_graph_3(infos, city, prob_out, prob_new):
    '''
    With this function, we are simulating the responses we obtained from each of the surveyed ecosystems/cities. 
    The information we have is, for each evaluator, how many 
    
    Generator of a random graph, given the number of respondents of the questionnaire,
    the number of evaluators of collaborations, and the number of collaborations reported by each of them.
    
    Input:
     - infos : dictionary mapping from city to num_collab information
     - city : name of city to be simulated
     - prob_out : probability that a mentionned collaboration is outside of the network of evaluators
     - prob_new : probability that a mentionned collaboration is a newly mentionned org
     
    '''
    
    G=nx.DiGraph()
    setedges = set()
    
    
    #degrees of nodes inside and outside respondent list
    inside = dict()
    outside = dict()
    
    
    #determine number of organizations that answered and number of evaluators: 
    n_responded = int(muestra[city][0])
    n_evals = len(list(infos[city]['Colabs']))
    last_index = n_responded-1
    
    
    #number of total nodes, and number of edges:                  #still don't know how to use
    num_nodes = int(muestra[city][1])
    num_edges = int(muestra[city][2])
    
    for i in range(last_index+1):              #add all evaluators to the graph
        G.add_node(i)
        if i in range(n_evals):
            inside[i] = 0
        else:
            outside[i] = 0
        
    
    for i in inside.keys():
        
        deg_i = infos[city]['Colabs'][i]         # the degree of that node is in the info retrieved
        for j in range(deg_i):
            
            if random.random() < prob_out:           #edge added outside evaluators' list
                if outside == dict():                              # if list outside evaluators is new
                    last_index+=1
                    to = last_index
                    G.add_node(to)
                    G.add_edge(i,to)
                    setedges.add((i,to))
                    outside[to] = 1
                    inside[i] += 1
                else:
                    if random.random()<prob_new:               # if edge goes to a new vertex
                        last_index+=1       
                        to=last_index
                        G.add_node(to)
                        G.add_edge(i,to)
                        setedges.add((i,to))
                        outside[to] = 1
                        inside[i] +=1
                        
                    else:
                        vertices, degrees = [], []
                        for vertex, degree in outside.items():
                            if vertex!=i:
                                vertices.append(vertex)
                                degrees.append(degree+1)
                        s = sum(degrees)
                        w = [x/s for x in degrees]
                        to = choice(vertices, size=1, p=w)[0]
                        #while (i,to) in G.edges():
                        #while are_adjacent(i,to,G):
                        #    to = choice(vertices, size=1, p=w)
                        G.add_node(to)
                        G.add_edge(i,to)
                        setedges.add((i,to))
                        outside[to]+=1
                        inside[i]+=1
                        
            else:
                vertices, degrees = [], []
                for vertex, degree in inside.items():
                    if vertex != i:
                        vertices.append(vertex)
                        degrees.append(degree+1)
                s = sum(degrees)
                w = [x/s for x in degrees]
                to = choice(vertices, size=1, p=w)[0]
                #while (i,to) in G.edges():
                #while are_adjacent(i,to,G):
                #    to = choice(vertices, size=1, p=w)
                G.add_node(to)
                G.add_edge(i,to)
                setedges.add((i,to))
                inside[to]+=1
                inside[i]+=1
                
    #now we have the degree of each evaluator covered, we covered some nodes outside of the evaluator set. 
    available_nodes = n_responded - n_evals
    remaining_nodes = num_nodes - len(list(G.nodes()))
    remaining_edges = num_edges - len(list(G.edges()))

    if remaining_nodes <= 0:
        return G
    if remaining_edges <= 0:
        return G
    
    for i in range( last_index+1, num_nodes ):
        G.add_node(i)
    
    for i in range(n_responded):
        for j in range(3):

            if random.random()<prob_out:           #edge added outside evaluators' list
                if random.random()<prob_new:               # if edge goes to a new vertex
                    last_index+=1       
                    to=last_index
                    G.add_node(to)
                    G.add_edge(i,to)
                    setedges.add((i,to))
                    outside[to] = 1
                    if i in outside.keys():
                        outside[i]+=1  
                    elif i in inside.keys():
                        inside[i]+=1 
                        
                else:
                    vertices, degrees = [], []
                    for vertex, degree in outside.items():
                        if vertex!=i:
                            vertices.append(vertex)
                            degrees.append(degree+1)
                    s = sum(degrees)
                    w = [x/s for x in degrees]
                    to = choice(vertices, size=1, p=w)[0]
                    #while (i,to) in G.edges():
                    #while are_adjacent(i,to,G):
                    #    to = choice(vertices, size=1, p=w)
                    G.add_node(to)
                    G.add_edge(i,to)
                    setedges.add((i,to))
                    outside[to]+=1
                    if i in outside.keys():
                        outside[i]+=1  
                    elif i in inside.keys():
                        inside[i]+=1 
                        
            else:
                vertices, degrees = [], []
                for vertex, degree in inside.items():
                    if vertex != i:
                        vertices.append(vertex)
                        degrees.append(degree+1)
                s = sum(degrees)
                w = [x/s for x in degrees]
                to = choice(vertices, size=1, p=w)[0]
                #while (i,to) in G.edges():
                #while are_adjacent(i,to,G):
                #    to = choice(vertices, size=1, p=w)
                G.add_node(to)
                G.add_edge(i,to)
                setedges.add((i,to))
                inside[to]+=1
                if i in outside.keys():
                    outside[i]+=1  
                elif i in inside.keys():
                    inside[i]+=1  
    G=check_graph(G)
    return G
                
            
for city in list_cities:
    G= random_graph_3(cities_info,city,0.8, 0.6)
    nx.write_graphml(G,'../Random_Graphs_Third_Type_Corrected/Random_Graph_'+city+'.graphml')